In [1]:
import os
import subprocess
import platform
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont, ImageOps
import cv2
import numpy as np
from scipy.spatial import distance
import math
from datetime import datetime
import tempfile
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import gc  # For garbage collection

In [2]:
# Targeted Frame Extractor for RISO Contact Sheets

class TargetedFrameExtractor:
    """
    A specialized frame extractor designed specifically for contact sheets
    with registration marks at the four corners of each frame.
    """
    
    def __init__(self):
        self.debug_mode = False
        
    def set_debug_mode(self, debug=True):
        """Enable or disable debug mode."""
        self.debug_mode = debug
    
    def extract_frames(self, image_path, expected_frames_per_row=4):
        """
        Extract frames from a contact sheet using a targeted approach.
        
        Args:
            image_path: Path to the contact sheet image or CV2 image
            expected_frames_per_row: Expected number of frames per row (default: 4)
            
        Returns:
            List of extracted frame images
        """
        # Load the image
        if isinstance(image_path, str):
            image = cv2.imread(image_path)
        else:
            # Assume it's already a cv2 image
            image = image_path
            
        if image is None:
            print("Error: Could not load image")
            return []
            
        # Create a copy for visualization if debug mode is on
        debug_img = image.copy() if self.debug_mode else None
        
        # Get image dimensions
        img_height, img_width = image.shape[:2]
        
        # Step 1: Detect registration marks
        registration_marks = self._detect_registration_marks(image)
        
        if len(registration_marks) < 8:  # Need at least 8 marks (2 frames)
            print(f"Warning: Only {len(registration_marks)} registration marks detected. Need at least 8.")
            return []
            
        # Step 2: Identify grid structure
        grid_info = self._analyze_grid_structure(registration_marks, img_width, img_height, expected_frames_per_row)
        
        # Step 3: Extract frames based on grid
        frames = self._extract_frames_from_grid(image, grid_info, debug_img)
        
        # Save debug image if in debug mode
        if self.debug_mode and debug_img is not None:
            cv2.imwrite("debug_targeted_extraction.png", debug_img)
            
        print(f"Successfully extracted {len(frames)} frames")
        return frames
    
    def _detect_registration_marks(self, image):
        """
        Detect registration marks in the image.
        Specifically optimized for circle+crosshair registration marks.
        
        Args:
            image: Input image
            
        Returns:
            List of (x, y) coordinates of detected registration marks
        """
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Apply adaptive thresholding to handle different brightness/contrast
        binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY_INV, 21, 5)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter contours to find circular shapes (registration marks)
        registration_marks = []
        
        for contour in contours:
            # Skip very small or very large contours
            area = cv2.contourArea(contour)
            if area < 10 or area > 500:
                continue
                
            # Check circularity
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
                
            circularity = 4 * math.pi * area / (perimeter * perimeter)
            
            # Circles have circularity close to 1
            if 0.5 < circularity < 1.2:
                # Get center of contour
                M = cv2.moments(contour)
                if M["m00"] > 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    registration_marks.append((cx, cy))
        
        # If we found too many potential marks, filter them further
        if len(registration_marks) > 500:
            # Convert to numpy array
            points = np.array(registration_marks)
            
            # Use DBSCAN clustering to group nearby points
            clustering = DBSCAN(eps=20, min_samples=1).fit(points)
            
            # Get cluster centers
            unique_marks = []
            unique_labels = set(clustering.labels_)
            
            for cluster_id in unique_labels:
                # Get points in this cluster
                cluster_mask = clustering.labels_ == cluster_id
                cluster_points = points[cluster_mask]
                
                # Calculate center of cluster
                center_x = int(np.mean(cluster_points[:, 0]))
                center_y = int(np.mean(cluster_points[:, 1]))
                unique_marks.append((center_x, center_y))
            
            registration_marks = unique_marks
        
        return registration_marks
    
    def _analyze_grid_structure(self, marks, img_width, img_height, expected_frames_per_row):
        """
        Analyze the grid structure of registration marks.
        
        Args:
            marks: List of (x, y) coordinates of registration marks
            img_width: Width of the image
            img_height: Height of the image
            expected_frames_per_row: Expected number of frames per row
            
        Returns:
            Dictionary with grid analysis results
        """
        # Sort marks by y-coordinate
        sorted_by_y = sorted(marks, key=lambda m: m[1])
        
        # Adaptive tolerance based on image size
        y_tolerance = img_height * 0.02  # 2% of image height
        
        # Group marks into rows
        rows = []
        current_row = [sorted_by_y[0]]
        
        for i in range(1, len(sorted_by_y)):
            if abs(sorted_by_y[i][1] - current_row[0][1]) < y_tolerance:
                # Same row
                current_row.append(sorted_by_y[i])
            else:
                # New row
                if len(current_row) >= expected_frames_per_row:
                    rows.append(sorted(current_row, key=lambda m: m[0]))  # Sort row by x-coordinate
                current_row = [sorted_by_y[i]]
        
        # Add the last row if it has enough marks
        if current_row and len(current_row) >= expected_frames_per_row:
            rows.append(sorted(current_row, key=lambda m: m[0]))
        
        # Filter rows to ensure they have similar numbers of marks
        # This helps eliminate rows that might just be noise
        if rows:
            # Find the most common row length
            row_lengths = [len(row) for row in rows]
            most_common_length = max(set(row_lengths), key=row_lengths.count)
            
            # Keep only rows with length close to the most common length
            filtered_rows = [row for row in rows if abs(len(row) - most_common_length) <= 1]
            rows = filtered_rows
        
        # Calculate average distance between marks in each row
        x_distances = []
        for row in rows:
            for i in range(len(row) - 1):
                x_distances.append(row[i+1][0] - row[i][0])
        
        # Calculate average distance between rows
        y_distances = []
        for i in range(len(rows) - 1):
            y_distances.append(rows[i+1][0][1] - rows[i][0][1])
        
        # Calculate average mark spacing
        avg_x_distance = np.mean(x_distances) if x_distances else 0
        avg_y_distance = np.mean(y_distances) if y_distances else 0
        
        return {
            "rows": rows,
            "avg_x_distance": avg_x_distance,
            "avg_y_distance": avg_y_distance,
            "expected_frames_per_row": expected_frames_per_row
        }
    
    def _extract_frames_from_grid(self, image, grid_info, debug_img=None):
        """
        Extract frames based on the grid structure.
        
        Args:
            image: Input image
            grid_info: Grid analysis results
            debug_img: Optional image for debug visualization
            
        Returns:
            List of extracted frame images
        """
        rows = grid_info["rows"]
        frames = []
        
        # Calculate frame dimensions based on mark spacing
        avg_x_distance = grid_info["avg_x_distance"]
        avg_y_distance = grid_info["avg_y_distance"]
        
        # For each row except the last one
        for row_idx in range(len(rows) - 1):
            top_row = rows[row_idx]
            bottom_row = rows[row_idx + 1]
            
            # Skip if rows don't have enough marks
            if len(top_row) < 2 or len(bottom_row) < 2:
                continue
            
            # For each pair of marks in the top row (except the last one)
            for col_idx in range(len(top_row) - 1):
                # Skip if we don't have enough marks in the bottom row
                if col_idx >= len(bottom_row) - 1:
                    continue
                
                # Get the four corners of this frame
                top_left = top_row[col_idx]
                top_right = top_row[col_idx + 1]
                bottom_left = bottom_row[col_idx]
                bottom_right = bottom_row[col_idx + 1]
                
                # Calculate frame coordinates with padding
                padding = 5
                x = top_left[0] + padding
                y = top_left[1] + padding
                w = top_right[0] - top_left[0] - 2 * padding
                h = bottom_left[1] - top_left[1] - 2 * padding
                
                # Skip invalid frames
                if w <= 20 or h <= 20 or x < 0 or y < 0 or x + w >= image.shape[1] or y + h >= image.shape[0]:
                    print(f"Skipping invalid frame at ({x},{y},{w},{h})")
                    continue
                
                try:
                    # Extract the frame
                    frame = image[y:y+h, x:x+w].copy()
                    
                    # Verify frame is valid
                    if frame is None or frame.size == 0 or frame.shape[0] <= 0 or frame.shape[1] <= 0:
                        print(f"Skipping empty frame at ({x},{y},{w},{h})")
                        continue
                    
                    # Add to results
                    frames.append(frame)
                    
                    # Draw on debug image
                    if debug_img is not None:
                        cv2.rectangle(debug_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                        # Add frame number
                        frame_num = len(frames)
                        cv2.putText(debug_img, str(frame_num), (x+10, y+30), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                except Exception as e:
                    print(f"Error extracting frame at ({x},{y},{w},{h}): {str(e)}")
                    continue
        
        return frames

In [3]:
class AnimationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Animation Extractor")
        
        # Set up variables
        self.contact_sheet = None
        self.extracted_frames = []
        self.current_frame_index = 0
        self.is_playing = False
        self.play_after_id = None
        
        # Create UI variables
        self.status_var = tk.StringVar(value="Ready")
        self.frame_rate = tk.DoubleVar(value=12.0)  # Default 12 FPS
        self.frames_per_row = tk.IntVar(value=4)  # Default 4 frames per row
        self.extraction_mode = tk.StringVar(value="auto")  # Default to auto mode
        self.registration_mark_threshold = tk.IntVar(value=70)  # Default 70%
        self.debug_mode = tk.BooleanVar(value=False)  # Debug mode toggle
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create left panel for controls
        control_frame = ttk.Frame(main_frame, padding="5", width=200)
        control_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        
        # Create right panel for image display
        display_frame = ttk.Frame(main_frame)
        display_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Create canvas for image display
        self.canvas = tk.Canvas(display_frame, bg="#f0f0f0", highlightthickness=0)
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Input section
        input_frame = ttk.LabelFrame(control_frame, text="Input", padding="5")
        input_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(input_frame, text="Load Contact Sheet", command=self.load_contact_sheet).pack(fill=tk.X)
        ttk.Button(input_frame, text="Extract Frames", command=self.extract_frames_from_registration_marks).pack(fill=tk.X, pady=(5, 0))
        
        # Extraction mode section
        mode_frame = ttk.LabelFrame(control_frame, text="Extraction Settings", padding="5")
        mode_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Mode selection
        ttk.Label(mode_frame, text="Mode:").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="Auto", variable=self.extraction_mode, value="auto").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="Digital", variable=self.extraction_mode, value="digital").pack(anchor=tk.W)
        ttk.Radiobutton(mode_frame, text="RISO Print", variable=self.extraction_mode, value="riso").pack(anchor=tk.W)
        
        # Frames per row setting
        ttk.Label(mode_frame, text="Frames Per Row:").pack(anchor=tk.W, pady=(5, 0))
        ttk.Spinbox(mode_frame, from_=1, to=20, textvariable=self.frames_per_row, width=5).pack(anchor=tk.W)
        
        # Registration mark threshold
        ttk.Label(mode_frame, text="Registration Mark Threshold (%):").pack(anchor=tk.W, pady=(5, 0))
        ttk.Spinbox(mode_frame, from_=10, to=90, textvariable=self.registration_mark_threshold, width=5).pack(anchor=tk.W)
        
        # Debug mode toggle
        ttk.Checkbutton(mode_frame, text="Debug Mode", variable=self.debug_mode).pack(anchor=tk.W, pady=(5, 0))
        
        # Extract button
        ttk.Button(mode_frame, text="Extract Frames", command=self.extract_frames_from_registration_marks).pack(fill=tk.X, pady=(5, 0))
        
        # Playback controls
        playback_frame = ttk.LabelFrame(control_frame, text="Playback", padding="5")
        playback_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Frame rate control
        ttk.Label(playback_frame, text="Frame Rate (FPS):").pack(anchor=tk.W)
        ttk.Spinbox(playback_frame, from_=1, to=60, textvariable=self.frame_rate, width=5).pack(anchor=tk.W, pady=(0, 5))
        
        # Playback buttons
        button_frame = ttk.Frame(playback_frame)
        button_frame.pack(fill=tk.X)
        
        ttk.Button(button_frame, text="⏮", width=3, command=self.go_to_first_frame).pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏪", width=3, command=self.previous_frame).pack(side=tk.LEFT)
        self.play_button = ttk.Button(button_frame, text="▶", width=3, command=self.toggle_playback)
        self.play_button.pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏩", width=3, command=self.next_frame).pack(side=tk.LEFT)
        ttk.Button(button_frame, text="⏭", width=3, command=self.go_to_last_frame).pack(side=tk.LEFT)
        
        # Export section
        export_frame = ttk.LabelFrame(control_frame, text="Export", padding="5")
        export_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(export_frame, text="Export as GIF", command=self.export_as_gif).pack(fill=tk.X)
        ttk.Button(export_frame, text="Export as MP4", command=self.export_as_mp4).pack(fill=tk.X, pady=(5, 0))
        ttk.Button(export_frame, text="Export Frames", command=self.export_frames).pack(fill=tk.X, pady=(5, 0))
        
        # Status bar
        status_bar = ttk.Label(root, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Bind keyboard shortcuts
        self.root.bind("<Left>", lambda e: self.previous_frame())
        self.root.bind("<Right>", lambda e: self.next_frame())
        self.root.bind("<space>", lambda e: self.toggle_playback())
        
        # Initialize the display
        self.display_placeholder()
    
    def display_placeholder(self):
        """Display a placeholder message when no image is loaded"""
        self.canvas.delete("all")
        self.canvas.create_text(
            self.canvas.winfo_width() // 2 or 200,
            self.canvas.winfo_height() // 2 or 150,
            text="Load a contact sheet or animation frames to begin",
            fill="#666666",
            font=("Arial", 12)
        )
    
    def load_contact_sheet(self):
        """Load a contact sheet image"""
        # Use standard file dialog for all platforms to avoid issues
        file_path = filedialog.askopenfilename(
            title="Select Contact Sheet",
            filetypes=[
                ("Image files", "*.jpg *.jpeg *.png *.tif *.tiff"),
                ("PDF files", "*.pdf"),
                ("All files", "*.*")
            ]
        )
        
        if not file_path:
            return
        
        try:
            # Handle PDF files
            if file_path.lower().endswith(".pdf"):
                # Convert first page of PDF to image using a temporary file
                with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_file:
                    temp_path = temp_file.name
                
                # Use pdftoppm to convert PDF to PNG
                subprocess.run(["pdftoppm", "-png", "-singlefile", file_path, temp_path[:-4]])
                
                # Load the converted image
                self.contact_sheet = Image.open(temp_path)
                
                # Clean up the temporary file
                os.unlink(temp_path)
            else:
                # Load regular image file
                self.contact_sheet = Image.open(file_path)
            
            # Display the contact sheet
            self.display_contact_sheet()
            
            # Update status
            self.status_var.set(f"Loaded contact sheet: {os.path.basename(file_path)}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error loading file: {str(e)}")
    
    def display_contact_sheet(self):
        """Display the loaded contact sheet"""
        if self.contact_sheet is None:
            self.display_placeholder()
            return
        
        # Get canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        # If canvas hasn't been drawn yet, use default dimensions
        if canvas_width <= 1 or canvas_height <= 1:
            canvas_width = 600
            canvas_height = 400
        
        # Calculate scaling to fit the canvas
        img_width, img_height = self.contact_sheet.size
        scale = min(canvas_width / img_width, canvas_height / img_height)
        
        # Resize the image to fit the canvas
        new_width = int(img_width * scale)
        new_height = int(img_height * scale)
        resized_img = self.contact_sheet.resize((new_width, new_height), Image.LANCZOS)
        
        # Convert to PhotoImage for display
        self.tk_image = ImageTk.PhotoImage(resized_img)
        
        # Display on canvas
        self.canvas.delete("all")
        self.canvas.create_image(
            canvas_width // 2,
            canvas_height // 2,
            image=self.tk_image,
            anchor=tk.CENTER
        )
    
    def extract_frames_from_registration_marks(self):
        """
        Extract frames from a contact sheet using the targeted frame extractor.
        """
        if self.contact_sheet is None:
            messagebox.showwarning("Warning", "No contact sheet loaded")
            return
                
        try:
            # Update status
            self.status_var.set("Extracting frames... This may take a moment.")
            self.root.update()
            
            # Convert PIL Image to OpenCV format
            cv_image = cv2.cvtColor(np.array(self.contact_sheet), cv2.COLOR_RGB2BGR)
            
            # Create targeted frame extractor
            extractor = TargetedFrameExtractor()
            
            # Set debug mode if needed
            extractor.set_debug_mode(self.debug_mode.get())
            
            # Extract frames using the frames_per_row parameter
            frames = extractor.extract_frames(cv_image, self.frames_per_row.get())
            
            # Clean up to free memory
            del cv_image
            gc.collect()
            
            if not frames:
                messagebox.showwarning("Warning", "No frames could be extracted")
                return
            
            # Convert OpenCV frames back to PIL format
            pil_frames = []
            for frame in frames:
                # Verify frame is valid before conversion
                if frame is None or frame.size == 0 or frame.shape[0] <= 0 or frame.shape[1] <= 0:
                    print("Skipping invalid frame during conversion")
                    continue
                    
                try:
                    pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    pil_frames.append(pil_frame)
                    
                    # Clean up to free memory
                    del frame
                    gc.collect()
                except Exception as e:
                    print(f"Error converting frame: {str(e)}")
                    continue
            
            # Clean up to free memory
            del frames
            gc.collect()
            
            # Update the extracted frames
            self.extracted_frames = pil_frames
            self.current_frame_index = 0
            
            # Display the first frame if available
            if self.extracted_frames:
                self.display_current_frame()
                # Update status
                self.status_var.set(f"Successfully extracted {len(pil_frames)} frames")
            else:
                messagebox.showwarning("Warning", "No valid frames could be extracted")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error extracting frames: {str(e)}")
            import traceback
            traceback.print_exc()
    
    def display_current_frame(self):
        """Display the current frame"""
        if not self.extracted_frames:
            self.display_placeholder()
            return
        
        # Get the current frame
        frame = self.extracted_frames[self.current_frame_index]
        
        # Get canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        
        # If canvas hasn't been drawn yet, use default dimensions
        if canvas_width <= 1 or canvas_height <= 1:
            canvas_width = 600
            canvas_height = 400
        
        # Calculate scaling to fit the canvas
        img_width, img_height = frame.size
        scale = min(canvas_width / img_width, canvas_height / img_height)
        
        # Resize the image to fit the canvas
        new_width = int(img_width * scale)
        new_height = int(img_height * scale)
        resized_img = frame.resize((new_width, new_height), Image.LANCZOS)
        
        # Convert to PhotoImage for display
        self.tk_image = ImageTk.PhotoImage(resized_img)
        
        # Display on canvas
        self.canvas.delete("all")
        self.canvas.create_image(
            canvas_width // 2,
            canvas_height // 2,
            image=self.tk_image,
            anchor=tk.CENTER
        )
        
        # Display frame number
        self.canvas.create_text(
            10, 10,
            text=f"Frame {self.current_frame_index + 1}/{len(self.extracted_frames)}",
            fill="white",
            font=("Arial", 12),
            anchor=tk.NW
        )
        
        # Update status
        self.status_var.set(f"Displaying frame {self.current_frame_index + 1} of {len(self.extracted_frames)}")
    
    def next_frame(self):
        """Display the next frame"""
        if not self.extracted_frames:
            return
        
        self.current_frame_index = (self.current_frame_index + 1) % len(self.extracted_frames)
        self.display_current_frame()
    
    def previous_frame(self):
        """Display the previous frame"""
        if not self.extracted_frames:
            return
        
        self.current_frame_index = (self.current_frame_index - 1) % len(self.extracted_frames)
        self.display_current_frame()
    
    def go_to_first_frame(self):
        """Go to the first frame"""
        if not self.extracted_frames:
            return
        
        self.current_frame_index = 0
        self.display_current_frame()
    
    def go_to_last_frame(self):
        """Go to the last frame"""
        if not self.extracted_frames:
            return
        
        self.current_frame_index = len(self.extracted_frames) - 1
        self.display_current_frame()
    
    def toggle_playback(self):
        """Toggle animation playback"""
        if not self.extracted_frames:
            messagebox.showinfo("Info", "No frames to play")
            return
        
        # If there's only one frame, playback doesn't make sense
        if len(self.extracted_frames) <= 1:
            messagebox.showinfo("Info", "Animation requires multiple frames for playback")
            return
        
        # Toggle playback state
        self.is_playing = not self.is_playing
        
        # Update button text
        self.play_button.config(text="⏸" if self.is_playing else "▶")
        
        # Start or stop playback
        if self.is_playing:
            self.play_animation()
        else:
            # Cancel any scheduled playback
            if self.play_after_id:
                self.root.after_cancel(self.play_after_id)
                self.play_after_id = None
    
    def play_animation(self):
        """Play the animation"""
        if not self.is_playing or not self.extracted_frames:
            return
        
        try:
            # Display the current frame
            self.display_current_frame()
            
            # Move to the next frame
            self.current_frame_index = (self.current_frame_index + 1) % len(self.extracted_frames)
            
            # Calculate delay based on frame rate
            # Make sure frame rate is at least 1 FPS to avoid division by zero
            frame_rate = max(1.0, self.frame_rate.get())
            delay_ms = int(1000 / frame_rate)
            
            # Schedule the next frame
            self.play_after_id = self.root.after(delay_ms, self.play_animation)
            
        except Exception as e:
            print(f"Error during playback: {str(e)}")
            self.is_playing = False
            self.play_button.config(text="▶")
    
    def export_as_gif(self):
        """Export the animation as a GIF"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        try:
            # Use standard file dialog
            file_path = filedialog.asksaveasfilename(
                title="Save GIF Animation",
                defaultextension=".gif",
                filetypes=[("GIF files", "*.gif"), ("All files", "*.*")]
            )
            
            if not file_path:
                return
            
            # Ensure file has .gif extension
            if not file_path.lower().endswith(".gif"):
                file_path += ".gif"
            
            # Get export parameters
            fps = self.frame_rate.get()
            
            # Calculate frame duration in milliseconds
            duration = int(1000 / fps)
            
            # Update status
            self.status_var.set("Exporting GIF... This may take a moment.")
            self.root.update()
            
            # Save as GIF
            self.extracted_frames[0].save(
                file_path,
                save_all=True,
                append_images=self.extracted_frames[1:],
                optimize=False,
                duration=duration,
                loop=0  # 0 means loop forever
            )
            
            # Update status
            self.status_var.set(f"Exported GIF animation to {file_path}")
            
            # Show success message
            messagebox.showinfo("Success", f"Animation exported to {file_path}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error exporting GIF: {str(e)}")
    
    def export_as_mp4(self):
        """Export the animation as an MP4 video"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        try:
            # Use standard file dialog
            file_path = filedialog.asksaveasfilename(
                title="Save MP4 Video",
                defaultextension=".mp4",
                filetypes=[("MP4 files", "*.mp4"), ("All files", "*.*")]
            )
            
            if not file_path:
                return
            
            # Ensure file has .mp4 extension
            if not file_path.lower().endswith(".mp4"):
                file_path += ".mp4"
            
            # Get export parameters
            fps = self.frame_rate.get()
            
            # Get dimensions of first frame
            width, height = self.extracted_frames[0].size
            
            # Update status
            self.status_var.set("Exporting MP4... This may take a moment.")
            self.root.update()
            
            # Create a temporary directory for frame images
            with tempfile.TemporaryDirectory() as temp_dir:
                try:
                    # Save frames as temporary images
                    frame_paths = []
                    for i, frame in enumerate(self.extracted_frames):
                        frame_path = os.path.join(temp_dir, f"frame_{i:04d}.png")
                        frame.save(frame_path)
                        frame_paths.append(frame_path)
                    
                    # Create video writer
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 codec
                    video_writer = cv2.VideoWriter(file_path, fourcc, fps, (width, height))
                    
                    # Add each frame to the video
                    for frame_path in frame_paths:
                        img = cv2.imread(frame_path)
                        if img is not None and img.size > 0:
                            video_writer.write(img)
                        else:
                            print(f"Warning: Could not read frame from {frame_path}")
                    
                    # Release the video writer
                    video_writer.release()
                    
                    # Update status
                    self.status_var.set(f"Exported MP4 video to {file_path}")
                    
                    # Show success message
                    messagebox.showinfo("Success", f"Video exported to {file_path}")
                    
                except Exception as e:
                    raise Exception(f"Error creating video: {str(e)}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error exporting MP4: {str(e)}")
    
    def export_frames(self):
        """Export individual frames as image files"""
        if not self.extracted_frames:
            messagebox.showwarning("Warning", "No frames to export")
            return
        
        try:
            # Use standard folder dialog
            export_dir = filedialog.askdirectory(title="Select Export Directory")
            
            if not export_dir:
                return
            
            # Update status
            self.status_var.set("Exporting frames... This may take a moment.")
            self.root.update()
            
            # Export frames
            successful_exports = 0
            for i, frame in enumerate(self.extracted_frames):
                try:
                    frame_path = os.path.join(export_dir, f"frame_{i+1:04d}.png")
                    frame.save(frame_path)
                    successful_exports += 1
                except Exception as e:
                    print(f"Error saving frame {i+1}: {str(e)}")
            
            # Update status
            self.status_var.set(f"Exported {successful_exports} frames to {export_dir}")
            
            # Show success message
            messagebox.showinfo("Success", f"Exported {successful_exports} frames to {export_dir}")
            
        except Exception as e:
            messagebox.showerror("Error", f"Error exporting frames: {str(e)}")

In [4]:
# Create and run the application
root = tk.Tk()
app = AnimationApp(root)

# Set window size
root.geometry("800x600")

# Start the main loop
root.mainloop()

2025-03-25 16:56:47.108 Python[20468:1061728] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-25 16:56:47.108 Python[20468:1061728] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Skipping invalid frame at (579,298,18,29)
Skipping invalid frame at (607,316,59,12)
Skipping invalid frame at (676,308,170,17)
Skipping invalid frame at (1048,327,119,2)
Skipping invalid frame at (1177,327,202,13)
Skipping invalid frame at (1389,324,19,5)
Skipping invalid frame at (1418,326,9,14)
Skipping invalid frame at (197,350,-10,478)
Skipping invalid frame at (510,349,-8,459)
Skipping invalid frame at (835,379,2,467)
Skipping invalid frame at (655,840,105,14)
Skipping invalid frame at (860,858,109,7)
Skipping invalid frame at (979,821,0,74)
Skipping invalid frame at (989,856,30,10)
Skipping invalid frame at (1134,856,213,-2)
Skipping invalid frame at (1397,816,3,52)
Skipping invalid frame at (1410,838,-6,18)
Skipping invalid frame at (1414,859,35,7)
Skipping invalid frame at (197,903,-7,59)
Skipping invalid frame at (508,875,-5,98)
Skipping invalid frame at (833,876,2,97)
Skipping invalid frame at (1157,904,-8,102)
Skipping invalid frame at (1475,876,-9,134)
Skipping invalid fram